In [1]:
# Module and data import
import numpy as np
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from master_scripts.classes import DSNT
from master_scripts.data_functions import (load_experiment, get_git_root, separation_distance, energy_difference,
                                           relative_energy, event_indices, normalize_image_data, import_real_data)
from master_scripts.analysis_functions import (doubles_classification_stats, singles_classification_stats)
from master_scripts.plotting import plot_roc_curve
from sklearn.metrics import f1_score, roc_curve
%load_ext autoreload
%autoreload 2
repo_root = get_git_root()
FIG_PATH = repo_root + "figures/"

In [2]:
DATA_PATH = repo_root + "data/real/"
DATA_FILENAME = "anodedata_500k.txt"
MODEL_PATH = "../../models/"
RESULTS_PATH =  "../../results/"
classifiers = {
    'logistic': "17e010a5e6a4",
    'dense_10' : "e217c356f987",
    'dense_100_5' : "68f272ec5935",
    'cnn_shallow' : "ba147c2c6841",
    'cnn_deep' : "556d470b165e",
}

In [3]:
# Load initial events and images
events, images = import_real_data(DATA_PATH + DATA_FILENAME) # images not normalized
#images = normalize_image_data(images)


In [4]:
# load classification results (doesn't contain images)
results = {}

for k in classifiers.keys():
    with open(RESULTS_PATH + "events_classified_" + DATA_FILENAME[:-4] 
              + "_C_" + k
              + ".json") as fp:
        results[k] = json.load(fp)

In [5]:
# Load experiments
experiments = {}
for k in classifiers.keys():
    experiments[k] = load_experiment(classifiers[k])

In [9]:
for k, events in results.items():
    decays = [key for key in events.keys() if events[key]['event_descriptor'] == 2]
    singles = []
    doubles = []
    for key in decays:
        if events[key]['event_class'] == 'single': singles.append(key)
        if events[key]['event_class'] == 'double': doubles.append(key)
            
    print("{:12s}: fraction (singles, doubles) = ({:.2f}, {:.2f})".format(
        k,
        len(singles)/(len(singles) + len(doubles)),
        len(doubles)/(len(singles) + len(doubles)),
        )
    )

logistic    : fraction (singles, doubles) = (0.79, 0.21)
dense_10    : fraction (singles, doubles) = (0.80, 0.20)
dense_100_5 : fraction (singles, doubles) = (0.27, 0.73)
cnn_shallow : fraction (singles, doubles) = (0.31, 0.69)
cnn_deep    : fraction (singles, doubles) = (0.12, 0.88)
